<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# LinkedIn - Send connections to Google Sheets spreadsheet

**Tags:** #linkedin #google #sheets #spreadsheet #connections #python

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Last update:** 2023-10-16 (Created: 2023-10-16)

**Description:** This notebook will show how to send LinkedIn connections to a Google Sheets spreadsheet. It is usefull for organizations to keep track of their contacts and build relationships.

**References:**
- [LinkedIn API Documentation](https://docs.microsoft.com/en-us/linkedin/shared/references/v2/profile/profile-views?context=linkedin/consumer/context)
- [Google Sheets API Documentation](https://developers.google.com/sheets/api/reference/rest/)

## Input

### Import libraries

In [ ]:
import requests
import json
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

### Setup variables
- `access_token`: Access token to access LinkedIn API. [Link to procedure](https://docs.microsoft.com/en-us/linkedin/shared/authentication/authorization-code-flow?context=linkedin/consumer/context)
- `google_credentials`: Credentials to access Google Sheets API. [Link to procedure](https://gspread.readthedocs.io/en/latest/oauth2.html)

In [ ]:
access_token = "<access_token>"
google_credentials = {
    "type": "service_account",
    "project_id": "<project_id>",
    "private_key_id": "<private_key_id>",
    "private_key": "<private_key>",
    "client_email": "<client_email>",
    "client_id": "<client_id>",
    "auth_uri": "https://accounts.google.com/o/oauth2/auth",
    "token_uri": "https://oauth2.googleapis.com/token",
    "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
    "client_x509_cert_url": "<client_x509_cert_url>",
}

## Model

### Get LinkedIn connections

Long description of the function: This function will get the connections from the LinkedIn API and store them in a Pandas DataFrame.

In [ ]:
def get_connections():
    headers = {"Authorization": "Bearer " + access_token}
    url = "https://api.linkedin.com/v2/connections"
    response = requests.get(url, headers=headers)
    response_json = json.loads(response.text)
    connections = response_json["elements"]
    df = pd.DataFrame(connections)
    return df

### Send connections to Google Sheets

Long description of the function: This function will send the connections from the Pandas DataFrame to a Google Sheets spreadsheet.

In [ ]:
def send_connections_to_google_sheets(df):
    scope = [
        "https://spreadsheets.google.com/feeds",
        "https://www.googleapis.com/auth/drive",
    ]
    credentials = ServiceAccountCredentials.from_json_keyfile_dict(
        google_credentials, scope
    )
    gc = gspread.authorize(credentials)
    spreadsheet = gc.open("LinkedIn Connections")
    worksheet = spreadsheet.get_worksheet(0)
    worksheet.clear()
    worksheet.update([df.columns.values.tolist()] + df.values.tolist())

## Output

### Display result

In [ ]:
connections_df = get_connections()
send_connections_to_google_sheets(connections_df)